In [1]:
print 'Hello World!'

Hello World!


In [ ]:
>>>>>>>>>>>>>>>>>>Definition<<<<<<<<<<<<<<<<
User clickstream data and information about a group of hotels is given. The users are classified into a set of classes.
The classes are, Backpackers, Family, Couple. The objective is to predict the segment for a new customer using a
classifier. The task is to explore different classifiers and then select the best one for predictions. Cross-validation
is used to test the models by determining the F1 score.GraidCV and RandomCV are used to determine optimum
hyper parameters for the classifiers.
>>Data<<
Data is given in the form of two tables.


The exercise is done  based on the following steps, <Step-1> Data mining and extraction:- This step involves reading the train_search.csv and hotel.csv and converting the data into a single table, globalTrain.csv. This process is executed using "join" using python pandas. However, here due to the size of data, I used mysql to generate the table. <Step-2> Identifying features and target in the test and training data. The features are extracted from the globalTrain and the target is the "Segment". This step involves the python pandas library and the following dataframes are created: X_train (features of trainigng data), y_train (target of training data), X_test (features of test dat i.e. evaluation.csv) and y_test (target of test data i.e. our final objective, the y_test is generated at the end). <Step-3> Performs cross-validation based F1 score tests and build the classifier models. Then fit the training data in the classifier. <Step-4> Predict y_test from the classifier fits.

Read train_search.csv and store the data in a data frame named, "searchData"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Tell iPython to include plots inline in the notebook
%matplotlib inline
# Read dataset
searchData = pd.read_csv("train_search.csv")
#print "Dataset has {} rows, {} columns".format(*searchData.shape)
#print searchData.head() # print the first 5 rows
print "Successfull!!"

/home/akansha/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Read Hotel.csv and store the data in a data frame named, "hotelData"

In [3]:
# Read dataset
hotelData = pd.read_csv("Hotel.csv")
#print "Dataset has {} rows, {} columns".format(*hotelData.shape)
#print hotelData.head() # print the first 5 rows
print "Successfull!!"

Successfull!!


Now join the two data frames based on "HotelCOde" and develop a single data frame "globalData". in this process also create a csv file, "globalTrain.csv". Due to the size of searchData and computational limitations on my laptop I did not execute this join, rather used mysql to execute the join.

In [24]:
#globalData = searchData.join(hotelData, on='HotelCode')
#print "Dataset has {} rows, {} columns".format(*globalData.shape)
#print globalData.head() # print the first 5 rows


In [4]:
globalData = pd.read_csv("globalTrain.csv")
#print globalData.head() # print the first 5 rows
globalData=globalData.rename(columns = {' HotelCode':'HotelCode'})
globalData=globalData.rename(columns = {' Age':'Age'})
globalData=globalData.rename(columns = {' Gender':'Gender'})
globalData=globalData.rename(columns = {' Number of Rooms':'Number of Rooms'})
globalData=globalData.rename(columns = {' Check in date':'Check in date'})
globalData=globalData.rename(columns = {' Check Out Date':'Check Out Date'})
globalData=globalData.rename(columns = {' Seen Price':'Seen Price'})
globalData=globalData.rename(columns = {' isClicked':'isClicked'})
globalData=globalData.rename(columns = {' isBooked':'isBooked'})
globalData=globalData.rename(columns = {' Segment':'Segment'})
#print globalData.dtypes;
globalData['Booking Date'] =  pd.to_datetime(globalData['Booking Date'])
globalData['Check in date'] =  pd.to_datetime(globalData['Check in date'])
globalData['Check Out Date'] =  pd.to_datetime(globalData['Check Out Date'])
globalData['isClicked'] =  globalData['isClicked'].astype(str)
globalData['isBooked'] =  globalData['isBooked'].astype(str)

globalData['Stay Period'] = (globalData['Check Out Date'] - globalData['Check in date'])/np.timedelta64(1, 'D');
globalData['Travel Gap'] = (globalData['Check in date'] - globalData['Booking Date'])/np.timedelta64(1, 'D');
#print globalData.dtypes;
#print "Dataset has {} rows, {} columns".format(*globalData.shape)
#print globalData.head() # print the first 5 rows
print "Successfull!!"

Successfull!!


From the data, it is evident that the segment depends on,
1> Age, 2> Gender, 3> Number of rooms, 4> Seen price, 5> isCLicked, 6> isBooked, 7> Star Rating, 8> Trip Adviser Rating, 9> Stay period (Difference in days between Check out date and Check in date), 10 > Travel Gap  (Difference in days between booking date and check in date).
Therefore, there are 10 features as mentioned above. Some of the features have a string/object value, convert those to discretes ones an zeros. 

In [31]:
#Now separate training data
X_train=globalData.iloc[:,[3,4,5,8,9,10,16,17,18,19]];
#X_train=globalData.iloc[:,[3]];
X_train['Gender'] = X_train['Gender'].replace(['male', 'female'], [1, 0])
X_train['isClicked'] = X_train['isClicked'].replace(['True', 'False'], [1, 0])
X_train['isBooked'] = X_train['isBooked'].replace(['True', 'False'], [1, 0])
X_train[X_train < 0] = 0
#print X_train.head();
#print X_train.dtypes;
#print X_train['Travel Gap'];
print "Successful!!"

/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

Successful!!


In [6]:
# Import clustering modules
from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from sklearn.decomposition import PCA

In [33]:
pca = PCA(n_components=10)
pca.fit(X_train)
# Print the components and the amount of variance in the data contained in each dimension
#print pca.components_
print pca.explained_variance_ratio_

[  9.99999999e-01   1.05329442e-09   7.57287566e-11   1.22392571e-12
   2.25296960e-13   8.80978053e-14   8.45287761e-14   6.86137409e-14
   4.96749658e-14   1.11425796e-14]


In [30]:
# TODO: First we reduce the data to two dimensions using PCA to capture variation
pca = PCA(n_components=3)
reduced_data = pca.fit_transform(X_train)
print reduced_data[:10] #print upto 10 elements
print pca.components_
print pca.explained_variance_ratio_

ValueError: n_components=3 invalid for n_features=1

In [ ]:
# TODO: Implement your clustering algorithm here, and fit it to the reduced data for visualizati
# The visualizer below assumes your clustering object is named ’clusters’
clusters = KMeans(init='k-means++', n_clusters=3, n_init=10)
clusters.fit(reduced_data)
#clusters = ?
print clusters
KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
verbose=0)

In [16]:
#Now separate training data - Target
y_train=globalData.iloc[:,[11]];
y_train['Segment'] = y_train['Segment'].replace(['backpacker', 'couple','family'], [1, 3, 2])
#print y_train.head();
#print y_train.dtypes;
print "Successfull!!"

Successfull!!


/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


The best way to test a classifier model is cross-validation. In cross-validation an input set is brocken into training and testing set. The testting set is used to test the model built using the training set. And then the F1 score is computed. The next block of code is used to split the input data randomly into traing set and testing set. After the block is executed a set of training and test features and target are available for cross validation. The variables generated are X_train_cv, y_train_cv, X_test_cv and y_test_cv.

In [17]:
from sklearn.cross_validation import train_test_split
# First, decide how many training vs test samples you want
num_test = 40000
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_train, y_train, test_size=num_test)
print "Training set: {} samples".format(X_train_cv.shape[0])
print "Test set: {} samples".format(X_test_cv.shape[0])

Training set: 122848 samples
Test set: 40000 samples


The next block of code is a method that fits a training data into an inputted classifier and also returns and prints the time it takes to train the model.

In [18]:
import time
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return (end - start)


A decision tree classifier is initiated and a classifier model is built and fittedwith the cross-validation training features and target.

In [19]:
# Fit model to training data
#Best classifier
#clf = DecisionTreeClassifier(random_state=0)

from sklearn import svm, grid_search, naive_bayes

#SVC does not work
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 20]}
#svr = svm.SVC()
#clf = grid_search.GridSearchCV(svr, parameters)

#Bad results
#clf= GaussianNB()

#Works
#from sklearn.neighbors import KNeighborsClassifier
#clf = KNeighborsClassifier(n_neighbors=9)

#Works
from  sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,max_features=None)

#Works
#from  sklearn.ensemble import AdaBoostClassifier
#clf = AdaBoostClassifier(n_estimators=100,random_state=None)

#Works
#from  sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#clf = LinearDiscriminantAnalysis(solver='eigen', shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001)

#Works
#from  sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#clf = QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0, store_covariances=False, tol=0.0001)

train_classifier(clf, X_train_cv, y_train_cv) # note: using entire training set here
#print clf # you can inspect the learned model by printing it

Training RandomForestClassifier...
Done!
Training time (secs): 5.353


/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


5.352785110473633

The following block and method is used to predict target for an input features set. This also returns the f_score based on the predicted target and the actual target.

In [20]:
#Predict on training set and compute F1 score
from sklearn.metrics import f1_score
def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    #return f1_score(target.values, y_pred, pos_label='yes')
    return f1_score(target.values, y_pred) , (end - start)


The following block of code is used to see the f1 score of the training data used as a test set and the cross-validation test set.

In [21]:
#print y_train.head()
#print X_train.head()
train_cv_f1_score = predict_labels(clf, X_train_cv, y_train_cv)
print "F1 score for training set: {}".format(train_cv_f1_score)
# Predict on test data
test_cv_f1_score = predict_labels(clf, X_test_cv, y_test_cv)
print "F1 score for test set: {}".format(test_cv_f1_score)

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 1.165
F1 score for training set: (0.58545795721584037, 1.1648800373077393)
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.363

/home/akansha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/home/akansha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1


F1 score for test set: (0.58704401792242944, 0.3634068965911865)


In [22]:
globalTestData = pd.read_csv("globalTest.csv")
#print globalTestData.head() # print the first 5 rows
globalTestData=globalTestData.rename(columns = {' HotelCode':'HotelCode'})
globalTestData=globalTestData.rename(columns = {' Age':'Age'})
globalTestData=globalTestData.rename(columns = {' Gender':'Gender'})
globalTestData=globalTestData.rename(columns = {' Number of Rooms':'Number of Rooms'})
globalTestData=globalTestData.rename(columns = {' Check in date':'Check in date'})
globalTestData=globalTestData.rename(columns = {' Check Out Date':'Check Out Date'})
globalTestData=globalTestData.rename(columns = {' Seen Price':'Seen Price'})
globalTestData=globalTestData.rename(columns = {' isClicked':'isClicked'})
globalTestData=globalTestData.rename(columns = {' isBooked':'isBooked'})
globalTestData=globalTestData.rename(columns = {' Segment':'Segment'})
#print globalTestData.dtypes;
globalTestData['Booking Date'] =  pd.to_datetime(globalTestData['Booking Date'])
globalTestData['Check in date'] =  pd.to_datetime(globalTestData['Check in date'])
globalTestData['Check Out Date'] =  pd.to_datetime(globalTestData['Check Out Date'])
globalTestData['isClicked'] =  globalTestData['isClicked'].astype(str)
globalTestData['isBooked'] =  globalTestData['isBooked'].astype(str)

globalTestData['Stay Period'] = (globalTestData['Check Out Date'] - globalTestData['Check in date'])/np.timedelta64(1, 'D');
globalTestData['Travel Gap'] = (globalTestData['Check in date'] - globalTestData['Booking Date'])/np.timedelta64(1, 'D');
#print globalTestData.dtypes;
#print "Dataset has {} rows, {} columns".format(*globalTestData.shape)
#print globalTestData.head() # print the first 5 rows
print "Successfull!!"

Successfull!!


In [ ]:
def string_to_date(my_string):
    if '-' in my_string:
        return datetime.datetime.strptime(my_string, '%d-%m-%y')
    elif my_string.isdigit():
        return datetime.datetime.strptime(my_string, '%d%m%y')
    elif my_string.isalnum():
        return datetime.datetime.strptime(my_string, '%d%b%Y')

In [28]:
#Now separate training data
#X_test=globalTestData.iloc[:,[3,4,5,8,9,10,16,17,18,19]];
X_test=globalTestData.iloc[:,[3]];
X_test['Gender'] = X_test['Gender'].replace(['male', 'female'], [1, 0])
X_test['isClicked'] = X_test['isClicked'].replace(['True', 'False'], [1, 0])
X_test['isBooked'] = X_test['isBooked'].replace(['True', 'False'], [1, 0])
X_test[X_test < 0] = 0
#print X_test.head();
#print X_test.dtypes;
#print X_train['Travel Gap'];
print "Successfull!!"

   Age
0   38
1   38
2   38
3   38
4   38


KeyError: 'Gender'

In [25]:
# Fit model to training data
#Best classifier
#clf = DecisionTreeClassifier(random_state=0)
train_classifier(clf, X_train, y_train) # note: using entire training set here
y_pred = clf.predict(X_test)
#print "Dataset has {} rows, {} columns".format(*y_pred.shape)
#print y_pred.head()
print "Successfull!!"

Training RandomForestClassifier...
Done!
Training time (secs): 5.341
Successfull!!


/home/akansha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [136]:
#print y_pred
#y_pred = y_pred.reshape((-1,1))
#df_pred = pd.DataFrame({'Segment':y_pred[:,0]})
#print df_pred.head()
#print "Dataset has {} rows, {} columns".format(*df_pred.shape)

In [26]:
y_pred = y_pred.reshape((-1,1))
df_output= pd.DataFrame({'Search ID':globalTestData['Search ID'],'Segment':y_pred[:,0]})
print df_output.head()
print "Dataset has {} rows, {} columns".format(*df_output.shape)
df_output.to_csv('akansha_out6.csv',columns=None, header=False)

   Search ID  Segment
0    1000000        3
1    1000001        3
2    1000002        3
3    1000003        3
4    1000004        3
Dataset has 297321 rows, 2 columns
